In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
import warnings
warnings.filterwarnings('ignore')
from data_preprocessing_revise import data_processing, split_train_test, input_target, input_target_grouping,input_target_grouping_1d
import matplotlib.pyplot as plt
import scipy.signal as signal

def lowpass_filter(data, sr, cut_off, order):
    nyq = 0.5 * sr
    b, a = signal.butter(order, cut_off/nyq, btype = 'low')
    lp_df = signal.filtfilt(b, a, data)
    return lp_df

In [2]:
file_list = [i.replace('\\','/') for i in sorted(glob(f"c3d/*txt"))]
file_name = []
for i in file_list:
    file, _, _ = os.path.basename(i).split('.')
    file_name.append(file)

In [3]:
FORCE, ANGLE, JOINT = data_processing(file_list, file_name)

411it [00:15, 27.14it/s]


In [4]:
REAR = {}
LEAD = {}
for trial in FORCE:
    kh = FORCE[trial]['kh_frame'][0]
    fc = FORCE[trial]['fc_frame'][0]
    br = FORCE[trial]['br_frame'][0]
    
    rear = FORCE[trial][['REAR_FORCE_X','REAR_FORCE_Y','REAR_FORCE_Z']].iloc[kh:fc+1]
    lead = FORCE[trial][['LEAD_FORCE_X','LEAD_FORCE_Y','LEAD_FORCE_Z']].iloc[fc:br+1]
    
    for c in rear:
        rear[c] = lowpass_filter(rear[c], 1080, 60, 4) 
    for c in lead:
        lead[c] = lowpass_filter(lead[c], 1080, 60, 4)
       
    REAR[trial] = rear
    LEAD[trial] = lead

In [5]:
MEDIAN = {
    'LEAD' : {
        'LEAD_FORCE_X' : [],
        'LEAD_FORCE_Y' : [],
        'LEAD_FORCE_Z' : [],
    },
    'REAR' : {
        'REAR_FORCE_X' : [],
        'REAR_FORCE_Y' : [],
        'REAR_FORCE_Z' : [],
    }
}

for data, da in zip([LEAD, REAR], ['LEAD','REAR']):
    for trial in data:
        for c in data[trial].columns:
            
            lp_df = data[trial][c]         
            Fs = 1080
            t = np.arange(0, 1.0, 1/Fs) 
            
            # Welch의 방법을 사용하여 파워 스펙트럼 밀도(PSD) 계산
            frequencies, powers = signal.welch(lp_df, Fs, nperseg=1024)

            # 중앙주파수 찾기
            cumulative_power = np.cumsum(powers)
            total_power = cumulative_power[-1]
            central_freq_index = np.where(cumulative_power >= total_power / 2)[0][0]
            central_frequency = frequencies[central_freq_index]
            MEDIAN[da][c].append(central_frequency)

In [6]:
lead_median = {
               'LEAD_FORCE_X' : MEDIAN['LEAD']['LEAD_FORCE_X'],
               'LEAD_FORCE_Y' : MEDIAN['LEAD']['LEAD_FORCE_Y'], 
               'LEAD_FORCE_Z' : MEDIAN['LEAD']['LEAD_FORCE_Z']
               }
rear_median = {
               'REAR_FORCE_X' : MEDIAN['REAR']['REAR_FORCE_X'],
               'REAR_FORCE_Y' : MEDIAN['REAR']['REAR_FORCE_Y'], 
               'REAR_FORCE_Z' : MEDIAN['REAR']['REAR_FORCE_Z']
               }

median = pd.concat([pd.DataFrame(lead_median), pd.DataFrame(rear_median)],axis=1)
median.describe().round(2)

,LEAD_FORCE_X,LEAD_FORCE_Y,LEAD_FORCE_Z,REAR_FORCE_X,REAR_FORCE_Y,REAR_FORCE_Z
count,378.00,378.00,378.00,378.00,378.00,378.00
mean,7.34,11.68,9.43,1.62,2.29,1.64
std,2.84,5.79,4.09,0.22,1.03,0.25
min,3.48,3.48,3.48,1.05,1.05,1.05
25%,5.97,6.28,6.07,1.46,1.54,1.47
50%,6.51,12.34,7.01,1.61,1.85,1.62
75%,7.01,14.59,13.01,1.78,3.01,1.79
max,18.51,29.35,22.34,2.20,9.46,2.92
